In [1]:
import pandas as pd
import time

import os
import multiprocessing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests
import glob
import pickle
import fitz
from string import digits
# from Codes.Section_01_Data_Extraction_Preparation.file_preparation import download_file, rotate_pdf, pickle_pdf_xml
# from Codes.Section_01_Data_Extraction_Preparation.pdf_metadata import get_pdf_metadata

### 1. Download all PDFs

In [5]:
### Read all the names of the PDFs to be downloaded
path = "G:\\ESA_downloads\\Demo_Alignment_Sheets\\"
Index0_path = path + '\\Index2-All_PDFs.csv'

Index0 = pd.read_csv(Index0_path, encoding= 'unicode_escape')
print("Number of PDF Files to be downloaded: ", len(Index0))

Index0 = Index0.rename(columns={'Application Short Name': 'App_Short_Name'})
#Index0 = Index0.head(25)

Index0.head()

Number of PDF Files to be downloaded:  1914


,file_name,application_name,esa_folder_link,DocumentID,DataID,esa_download_link,DataID_pdf,Section,Application filing date,link to folder with ESA,...,short_name,Location_y,Commodity,Application Name,Application Type (NEB Act),Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index
0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C0,268706,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,268706.pdf,52.0,37697,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Ekwan,"Alberta, British Columbia, All",Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C1,268709,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,268709.pdf,52.0,37697,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Ekwan,"Alberta, British Columbia, All",Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0H8C2 - 13.0 EIA - Section 13.7 Wildlife Part 2,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C2,268712,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,268712.pdf,52.0,37697,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Ekwan,"Alberta, British Columbia, All",Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0H8C3 - 13.0 EIA - Section 13.8 to 13.13,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C3,269018,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,269018.pdf,52.0,37697,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Ekwan,"Alberta, British Columbia, All",Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0H8C4 - 13.1 App 13A - Alignment Sheets,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C4,269021,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,269021.pdf,52.0,37697,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Ekwan,"Alberta, British Columbia, All",Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
### Function to download the PDFs

def download_file(path, Index0):
    count = 0
    error_urls = []
    error_dataIDs = []
    dataID = ""

    # Iterating each row in the Index0 dataframe
    for index, row in Index0.iterrows():
        try:
            dataID = row['DataID']
            download_url = 'http://docs2.cer-rec.gc.ca/ll-eng/llisapi.dll?func=ll&objId=' + str(dataID) + '&objaction=download&viewType=1'
            r = requests.get(download_url)  # scraping the PDF file from the URL
            full_name = os.path.join(path + 'Data_Files_v2\\PDFs\\' + str(dataID) + '.pdf')
            
            with open(full_name, 'wb') as file:
                file.write(r.content)
            count = count + 1
            
            #print(dataID)
        except:
            # storing the error logs
            error_urls.append(download_url)
            error_dataIDs.append(dataID)
            print("error with file {}".format(row['File Name']))

    # creating and and saving the error logs dataframe
    df_scraping_errorlog = pd.DataFrame({'error_dataIDs': error_dataIDs,
                                         'error_urls': error_urls
                                         })
    df_scraping_errorlog.to_csv(path + 'Data_Files_v2\\Error_Logs\\ScrapingPDFErrorLogs.csv', index=False, encoding='utf-8-sig')
    
    print("Count of the PDFs to be downloaded: ", len(Index0))
    print("Files successfully downloaded: ", count)
    print("Count of Files with Errors: ", len(df_scraping_errorlog))
    return count


In [7]:
# Download files
count = download_file(path, Index0)
print("Files downloaded ", count)

Count of the PDFs to be downloaded:  1914
Files successfully downloaded:  1914
Count of Files with Errors:  0
Files downloaded  1914


In [15]:
pdf_dataID = []
for index, row in Index0.iterrows():
    pdf_dataID.append(row['DataID'])
    
print(len(pdf_dataID))
print(pdf_dataID[0])

1914
268706


### 2. Extract Features for each page of the PDFs downloaded 

In [8]:
def count_word_in_str(long_str, key):
    count = 0
    for word in long_str.split():
        if key in word:
            count = count + 1
    return(count)
#count_word_in_str("vibudh rocks dh dh ddh","dh")

In [9]:
def count_words_in_str(long_str, keys):
    count = 0
    for word in long_str.split():
        for key in keys:
            if key in word:
                count = count + 1
    return(count)
#count_words_in_str("vibudh rocks dh dh ddh",["dh","vi"])

In [10]:
def area_of_imgblocks(imgblocks):
    sum_areas = 0
    for imgblock in imgblocks:
        block_area = imgblock['width']* imgblock['height']
        sum_areas = sum_areas + block_area
    return(sum_areas)

In [11]:
s = 'abc123def456ghi789ersuit834678 dhfuaiwhbui34tr234 zero0'
remove_digits = str.maketrans('', '', digits)
res = s.translate(remove_digits)
res

'abcdefghiersuit dhfuaiwhbuitr zero'

In [16]:
def extract_features(dataIDs):
    words_in_page = []
    
    scale = []
    km_kilometers = []
    m = []
    metres = []
    scale_grp = []
    
    legend = []
    
    figure = []
    mapp = []
    alignment_sheet = []
    sheet = []
    figure_grp = []
    
    north = []
    n = []
    north_grp = []
    
    dataID_pageNo = []
    
    count = 0
    
    No_of_images = []
    Area_of_images = []
    cnt = 0
    
    dataID_l = []
    page_no = []
    paths_l = []
    
    error_files = []
    i = 0
    
    for dataID in dataIDs:
        pdf_path = path + 'Data_Files_v2\\PDFs\\' + str(dataID) + '.pdf'
        i = i+1 
        print("File Starting: {}. PDF {} out of {}".format(dataID, i, len(dataIDs)))     
        
        try:
        
            j = 0    
            doc = fitz.open(pdf_path)             
            #print("hey", str(len(doc)))
            for page in doc:  # iterate through the pages
                
                j = j+1 #Number of pages
                cnt = cnt + 1
                p = page.getText("dict")
            
                blocks = p["blocks"]
                imgblocks = [b for b in blocks if b["type"] == 1]
                No_of_images.append(len(imgblocks))
                Area_of_images.append(area_of_imgblocks(imgblocks))
                
            
                p = str(p).replace('<p>', '').replace('</p>','').replace(".",'').replace(",",'').replace('"','').lower()
                p = p.translate(remove_digits)
                
                words_lst = p.split()
                
                word_count = 0
                big_words = ""
                words = ""
                #print(pdf_path)
                for word in words_lst:
                    words  = words + " " + word
                    if len(word) > 3:
                        word_count = word_count + 1
                        big_words = big_words + " " + word
                        
                words_in_page.append(word_count)
                #print(pdf_path)
                
                sc_grp = 0 
                if "scale" in big_words:
                    scale.append(count_word_in_str(big_words, "scale"))
                    sc_grp = 1
                else:
                    scale.append(0)
                    
                if ("kilometre" in big_words or "kilometer" in big_words or "km " in p):
                    km_kilometers.append(count_words_in_str(p, ["kilometre", "kilometer", "km "]))
                    sc_grp = 1
                else:
                    km_kilometers.append(0)
                    
                if("m " in p):
                    m.append(count_word_in_str(p, "m "))
                else:
                    m.append(0)
                    
                if("metre" in big_words or "meter" in big_words):
                    metres.append(count_words_in_str(big_words, ["meter","metre"]))
                    sc_grp = 1
                else:
                    metres.append(0)
                    
                if sc_grp > 0:
                    scale_grp.append(1)
                else:
                    scale_grp.append(0)
                    
                
                
                if "legend" in big_words:
                    legend.append(count_word_in_str(big_words,"legend"))
                else:
                    legend.append(0)
                
                    
                    
                fig_grp = 0
                if "figure" in big_words:
                    figure.append(count_word_in_str(big_words,"figure"))
                    fig_grp = 1
                else:
                    figure.append(0)
                    
                if "map " in p:
                    mapp.append(count_word_in_str(p,"map "))
                    fig_grp = 1
                else:
                    mapp.append(0)
                      
                if "alignment sheet" in big_words:
                    alignment_sheet.append(1)
                    fig_grp = 1
                else:
                    alignment_sheet.append(0)
                    
                if "sheet" in big_words:
                    sheet.append(count_word_in_str(big_words,"sheet"))
                    fig_grp = 1
                else:
                    sheet.append(0)
                
                if fig_grp > 0:
                    figure_grp.append(1)
                else:
                    figure_grp.append(0)
                
                
                   
                if "north" in big_words:
                    north.append(count_word_in_str(big_words, "north"))
                    no_grp = 1
                else:
                    north.append(0)
                    
                if "n" in p:
                    n.append(count_word_in_str(p, " n "))
                    no_grp = 1
                else:
                    n.append(0)
                          
                dataID_l.append(dataID)
                page_no.append(j)
                paths_l.append(pdf_path)
                
        
        except:
            #if 1==0:
            print("Error Found")
            error_files.append(dataID)
            page_no.append(j)

        
    Features = pd.DataFrame({'scale' : scale, 
                           'km_kilometers' : km_kilometers, 
                           'm' : m, 
                           'metres' : metres, 
                           'scale_grp' : scale_grp, 
                           'legend' : legend, 
                           'figure' : figure, 
                           'mapp' : mapp, 
                           'alignment_sheet' : alignment_sheet, 
                           'sheet' : sheet, 
                           'figure_grp' : figure_grp, 
                           'north' : north, 
                           'n' : n, 
                           'words_in_page' : words_in_page,
                            'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images,
                           'dataID' : dataID_l, 
                            'pageNo': page_no, 
                             'paths_l': paths_l
                           #'Y_class' : Y_class
                           })
    pages_ref = pd.DataFrame({'DataIDs': dataID_l,
                            'pdf_paths' : paths_l, 
                            'Page_no': page_no})
     
       
    #print("Total Number of pages processed: {}".format(count))   
    return Features, pages_ref, error_files

In [17]:
#"hello " + 3
X_df, pages_ref, error_files = extract_features(pdf_dataID) 
#Features
#dataIDs
#error_files



File Starting: 268706. PDF 1 out of 1914


mupdf: cannot find page 3363 in page tree
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invali

File Starting: 268709. PDF 2 out of 1914


mupdf: cannot find page 3363 in page tree


File Starting: 268712. PDF 3 out of 1914
File Starting: 269018. PDF 4 out of 1914
File Starting: 269021. PDF 5 out of 1914
File Starting: 269024. PDF 6 out of 1914
File Starting: 269027. PDF 7 out of 1914
File Starting: 269030. PDF 8 out of 1914
File Starting: 269033. PDF 9 out of 1914
File Starting: 268930. PDF 10 out of 1914
File Starting: 269036. PDF 11 out of 1914
File Starting: 269039. PDF 12 out of 1914
File Starting: 268933. PDF 13 out of 1914
File Starting: 269042. PDF 14 out of 1914
File Starting: 268936. PDF 15 out of 1914
File Starting: 269045. PDF 16 out of 1914
File Starting: 268939. PDF 17 out of 1914
File Starting: 269048. PDF 18 out of 1914
File Starting: 3892112. PDF 19 out of 1914
File Starting: 3892242. PDF 20 out of 1914
File Starting: 3891472. PDF 21 out of 1914
File Starting: 3891895. PDF 22 out of 1914
File Starting: 3891896. PDF 23 out of 1914
File Starting: 3892113. PDF 24 out of 1914
File Starting: 3891801. PDF 25 out of 1914
File Starting: 3892114. PDF 26 out

...

File Starting: 3762635. PDF 1795 out of 1914
File Starting: 3767244. PDF 1796 out of 1914
File Starting: 3762512. PDF 1797 out of 1914
File Starting: 3762513. PDF 1798 out of 1914
File Starting: 3762429. PDF 1799 out of 1914
File Starting: 3762514. PDF 1800 out of 1914
File Starting: 3767165. PDF 1801 out of 1914
File Starting: 3767166. PDF 1802 out of 1914
File Starting: 3767167. PDF 1803 out of 1914
File Starting: 3767168. PDF 1804 out of 1914


mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

File Starting: 3767349. PDF 1805 out of 1914
File Starting: 3767350. PDF 1806 out of 1914
File Starting: 3762095. PDF 1807 out of 1914
File Starting: 3762430. PDF 1808 out of 1914
File Starting: 3767179. PDF 1809 out of 1914
File Starting: 3762106. PDF 1810 out of 1914
File Starting: 3762453. PDF 1811 out of 1914
File Starting: 3767252. PDF 1812 out of 1914
File Starting: 3762107. PDF 1813 out of 1914
File Starting: 3762642. PDF 1814 out of 1914
File Starting: 3762643. PDF 1815 out of 1914
File Starting: 3767505. PDF 1816 out of 1914
File Starting: 3767506. PDF 1817 out of 1914
File Starting: 895179. PDF 1818 out of 1914
File Starting: 895070. PDF 1819 out of 1914
File Starting: 895182. PDF 1820 out of 1914
File Starting: 894827. PDF 1821 out of 1914
File Starting: 895073. PDF 1822 out of 1914
File Starting: 895003. PDF 1823 out of 1914
File Starting: 895185. PDF 1824 out of 1914
File Starting: 895076. PDF 1825 out of 1914
File Starting: 895237. PDF 1826 out of 1914
File Starting: 8950

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

File Starting: 3974826. PDF 1905 out of 1914
File Starting: 4003703. PDF 1906 out of 1914
File Starting: 3410189. PDF 1907 out of 1914
File Starting: 3931508. PDF 1908 out of 1914
File Starting: 3969837. PDF 1909 out of 1914
File Starting: 3969838. PDF 1910 out of 1914
File Starting: 3970827. PDF 1911 out of 1914
File Starting: 3970828. PDF 1912 out of 1914
File Starting: 3970829. PDF 1913 out of 1914
File Starting: 3970830. PDF 1914 out of 1914


In [19]:
print("Total PDFs", len(pdf_dataID))
print("Total No. of Pages", len(X_df))
print("Total Error Files", len(error_files))

Total PDFs 1914
Total No. of Pages 88648
Total Error Files 0


In [24]:
print(len(X_df))
X_df.head()

88648


,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID,pageNo,paths_l
0,0,0,0,0,0,0,0,0,0,0,0,0,0,894,1,79376,268706,1,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
1,0,0,0,2,1,0,0,0,0,0,0,0,0,1419,1,79376,268706,2,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1470,1,79376,268706,3,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1477,1,79376,268706,4,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1461,1,79376,268706,5,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...


In [26]:
X_df.to_csv(path + "Data_Files_v2\\all_features.csv")

In [20]:
X_df_features = X_df.copy()
#X_df_features.drop(columns=['dataID_pageNo', 'result'], inplace=True)
X_df_features.drop(columns=['dataID', 'pageNo', 'paths_l'], inplace=True)
X_df_features.head()

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images
0,0,0,0,0,0,0,0,0,0,0,0,0,0,894,1,79376
1,0,0,0,2,1,0,0,0,0,0,0,0,0,1419,1,79376
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1470,1,79376
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1477,1,79376
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1461,1,79376


In [27]:
features = pd.read_csv(path + "Data_Files_v2\\all_features.csv", encoding= 'unicode_escape')
features.head()

,Unnamed: 0,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID,pageNo,paths_l
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,894,1,79376,268706,1,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
1,1,0,0,0,2,1,0,0,0,0,0,0,0,0,1419,1,79376,268706,2,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1470,1,79376,268706,3,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1477,1,79376,268706,4,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1461,1,79376,268706,5,G:\ESA_downloads\Demo_Alignment_Sheets\Data_Fi...
